In [33]:
from sklearn.model_selection import train_test_split
from pandas import DataFrame, read_csv
from sklearn import metrics
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import xgboost
from xgboost import plot_importance, XGBRegressor

import numpy as np
from math import sqrt

%matplotlib inline

In [34]:
project = 'house-prices'
version='v0.1'

In [35]:
fulltrain=read_csv('./train.csv',index_col=0)
test=read_csv('./test.csv',index_col=0)
fixed_seed=1234578416
train80, valid20 = train_test_split(fulltrain, test_size=0.2, random_state=fixed_seed)

In [36]:
train80 = train80.select_dtypes(include=np.number)
valid20 = valid20.select_dtypes(include=np.number)

In [4]:
for df in [train80, valid20]:
    for column in df.columns:
        if column not in ['TotalBsmtSF','GarageArea','GarageCars','OverallQual','GrLivArea', 'LotArea', 'SalePrice']:
            df.drop(column, axis=1, inplace=True)

/home/guillaume/.virtualenvs/House-Prices/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [37]:
target_column='SalePrice'
X_train = train80.drop(target_column, axis=1).values
y_train = train80[target_column].values
X_val = valid20.drop(target_column, axis=1).values
y_val = valid20[target_column].values

In [38]:
model = XGBRegressor()

In [39]:
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)

In [40]:
y_pred = model.predict(X_val)

In [41]:
diff = np.log(y_val)-np.log(y_pred)
log_RMSE = sqrt((diff*diff).mean())
log_RMSE

0.1423577202772698